## MODEL TRAINING
- Import Data and Required Packages

In [1]:
import numpy as np
import pandas as pd


## modelling 

from sklearn.model_selection import train_test_split


## preprocessing
from sklearn.preprocessing import OneHotEncoder,LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer




In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head()

,smartlocation,roomtype,price,minimumnights,availability365,numberofreviews,reviewscoresrating,cancellationpolicy
0,Amsterdam,Entire home/apt,600.0,2.0,74.0,31.0,89.0,strict
1,Amsterdam,Entire home/apt,175.0,2.0,259.0,15.0,99.0,strict
2,Amsterdam,Entire home/apt,125.0,4.0,0.0,1.0,100.0,flexible
3,Amsterdam,Entire home/apt,130.0,3.0,0.0,22.0,97.0,flexible
4,Amsterdam,Entire home/apt,80.0,2.0,326.0,16.0,78.0,moderate


In [4]:
df.tail()

,smartlocation,roomtype,price,minimumnights,availability365,numberofreviews,reviewscoresrating,cancellationpolicy
345367,Parkdale,Entire home/apt,79.0,1.0,97.0,12.0,95.0,flexible
345368,Yarraville,Entire home/apt,189.0,3.0,364.0,4.0,95.0,flexible
345369,Footscray,Private room,30.0,1.0,0.0,4.0,60.0,moderate
345370,Yarraville,Private room,42.0,7.0,358.0,9.0,93.0,moderate
345371,West Footscray,Private room,33.0,3.0,337.0,3.0,100.0,strict


In [5]:
## dependent and independent features

X = df.drop('price',axis=1)
y = df.price


print(f"shape of X: {X.shape}")
print(f"shape of y: {y.shape}")

shape of X: (345372, 7)
shape of y: (345372,)


In [9]:
cat_cols_ohe = ['roomtype','cancellationpolicy']
cat_cols_lb = 'smartlocation'

num_cols_st = ['minimumnights','numberofreviews']
num_cols_minmax = ['availability365','reviewscoresrating']


cat_cols_lb_trans = LabelEncoder()
cat_cols_ohe_trans = OneHotEncoder()

num_cols_minmax_trans = MinMaxScaler()
num_cols_st_trans = StandardScaler()


preprocessor = ColumnTransformer([
    
    # ('lbe',cat_cols_lb_trans,cat_cols_lb),
    ('ohe',cat_cols_ohe_trans,cat_cols_ohe),
    ('minmax',num_cols_minmax_trans,num_cols_minmax),
    ('ss',num_cols_st_trans,num_cols_st)
    
])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9)


print(f"shape of X_train: {X_train.shape}")
print(f"shape of X_test: {X_test.shape}\n")
print(f"shape of y_train: {y_train.shape}")
print(f"shape of y_test: {y_test.shape}")



shape of X_train: (241760, 7)
shape of X_test: (103612, 7)

shape of y_train: (241760,)
shape of y_test: (103612,)


In [7]:
lb = LabelEncoder()
lb.fit_transform(X[cat_cols_lb])

array([  93,   93,   93, ..., 1384, 4027, 3904])

In [10]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

ValueError: Found unknown categories ['Hampstead Heath', 'London fields hackney', 'Tuckombil', 'Palma de majorqye', '\x8c 尖沙咀', 'København n ', 'Greenhills', '08014 BARCELONA', 'Plaka - Athens', '香港旺角女人街（中信大厦）', 'Haddington Rd', 'Lamma Island ', 'Montréal-Outremont', 'East Finchley', 'anderlecht ', 'Royal Borough of Kensington and Chelsea ', 'ROXBURY CROSSING', 'Lee green ', 'Wien/', 'Brookyln', 'Barcelona El RAVAL ', 'Brooklyn/Williamsburg', 'Pina (Algaida)', 'Sai Kung ', 'Döbling', 'Brooklyn\nBrooklyn', "Shepherd's Bush", 'Cala Serena (Felanitx)', 'Manchaca', 'Kilternan', 'Colonia Sant Jordi', 'Balham ', 'Blue Knob', 'Siu Sai Wan', 'vanløse', 'Wattle Grove', 'Mission', 'Baile Atha Cliath', 'Mosman Sydney', 'Cala Serena', 'Streatham Hill', 'Paris 8è', 'Londýn', '치펀데일', 'London Fields', 'notting hill', 'Coburg (Melbourne)', 'BE', 'Penia', 'Van nuys', 'Endeavour Hills', 'Centurion Building', 'Dublin 7 ', 'London TW1 2AR', 'Ruffré', 'Ses Olleries', 'palma de Mallorca ', 'Fairy Hill', 'Sa